In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [7]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

In [8]:
fill_mode = lambda col: col.fillna(col.mode()[0])
train = train.apply(fill_mode, axis=0)
train.isna().sum()

date           0
weight         0
resp_1         0
resp_2         0
resp_3         0
              ..
feature_126    0
feature_127    0
feature_128    0
feature_129    0
ts_id          0
Length: 138, dtype: int64

In [9]:
train['action'] = ((train['weight'].values * \
                         (train['resp_1'] + train['resp_2'] + train['resp_3'] + train['resp_4'] + train['resp'])/5) > 0).astype('int')
train['action']

0          0
2          0
3          0
6          0
7          1
          ..
1862550    0
1862552    0
1862584    0
1862587    0
1862595    0
Name: action, Length: 1571415, dtype: int64

In [10]:
train = train.drop(['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4'], axis=1)

In [11]:
X_train = train.loc[train.date < 425].drop(['date', 'ts_id', 'action'], axis=1)
X_val = train.loc[train.date >= 425].drop(['date', 'ts_id', 'action'], axis=1) 
y_train = train.loc[train.date < 425]['action']
y_val = train.loc[train.date >= 425]['action']

In [12]:
from sklearn.preprocessing import QuantileTransformer

quant_transf = QuantileTransformer(n_quantiles=100, output_distribution='normal')
X_train = quant_transf.fit_transform(X_train)
X_val = quant_transf.fit_transform(X_val)

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=60)
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.fit_transform(X_val)

In [14]:
from sklearn.svm import LinearSVC

svc = LinearSVC()

svc.fit(X_train_pca, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [ ]:
y_pred = svc.predict(X_val_pca)
svc.score(X_val_pca, y_val)